In [20]:
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import NearestNeighbors
from sklearn.decomposition import PCA

def clean_numeric_values(df):
    # Função para limpar formatos numéricos inconsistentes
    for col in df.columns:
        if df[col].dtype == 'object':  # verificar se a coluna é do tipo objeto (string)
            df[col] = df[col].str.replace(',', '.')
    return df

def treinar_modelo_knn(file_path, n_neighbors=20, n_components=2):
    # Carregar dados com especificação do separador decimal
    df = pd.read_csv(file_path, decimal=',')

    # Limpar formatos numéricos inconsistentes
    df = clean_numeric_values(df)

    # Selecionar apenas as colunas numéricas relevantes para o KNN
    features = ['track_genre', 'popularity']

    # Normalizar os dados
    scaler = StandardScaler()
    df_scaled = scaler.fit_transform(df[features])

    # Aplicar PCA para reduzir a dimensionalidade
    pca = PCA(n_components=n_components)
    df_pca = pca.fit_transform(df_scaled)

    # Treinar o modelo KNN com os dados reduzidos
    knn = NearestNeighbors(n_neighbors=n_neighbors, algorithm='auto')
    knn.fit(df_pca)
    
    return df, knn, scaler, features, pca

def recomendar_musicas(track_name, artist_name, df, knn, scaler, features, pca):
    # Limpar e padronizar as strings de entrada
    track_name = track_name.strip().lower()
    artist_name = artist_name.strip().lower()
    
    # Encontrar o track_id da música baseado no nome da música e do artista
    mask = (df['track_name'].str.lower() == track_name) & (df['artists'].str.lower().str.contains(artist_name))
    filtered_df = df[mask]
    
    if filtered_df.empty:
        return pd.DataFrame()  # Retornar DataFrame vazio se não encontrar resultados

    track_id = filtered_df['track_id'].values[0]
    
    # Obter características da música base
    track_features = df[df['track_id'] == track_id][features]
    
    # Normalizar as características
    track_features_scaled = scaler.transform(track_features)
    
    # Aplicar PCA às características da música base
    track_features_pca = pca.transform(track_features_scaled)
    
    # Encontrar as músicas mais próximas
    distances, indices = knn.kneighbors(track_features_pca)
    
    # Retornar as músicas recomendadas e remover duplicatas
    recomendacoes = df.iloc[indices[0]].drop_duplicates(subset=['track_name', 'artists'])
    return recomendacoes

file_path = 'datasettest.csv'  # caminho do arquivo CSV
df, knn, scaler, features, pca = treinar_modelo_knn(file_path)

track_name = 'starboy'  # substitua pelo track_name da música base
artist_name = 'weeknd'  # substitua pelo artist_name da música base
recomendacoes = recomendar_musicas(track_name, artist_name, df, knn, scaler, features, pca)

if not recomendacoes.empty:
    print(recomendacoes[['track_name', 'artists', 'track_genre', 'album_name']])
else:
    print("Nenhuma música encontrada com o nome ou artista fornecido.")


                                    track_name                      artists  \
81803   Running Up That Hill (A Deal With God)                    Kate Bush   
81101                        I Was Never There     The Weeknd;Gesaffelstein   
81012                      MIDDLE OF THE NIGHT                   Elley Duhé   
81015                               Dandelions                      Ruth B.   
81050                        Until I Found You              Stephen Sanchez   
81004                                  Starboy         The Weeknd;Daft Punk   
81000                          Blinding Lights                   The Weeknd   
81064                 One Kiss (with Dua Lipa)       Calvin Harris;Dua Lipa   
81054                STAY (with Justin Bieber)  The Kid LAROI;Justin Bieber   
81053                     lovely (with Khalid)         Billie Eilish;Khalid   
81200                          Save Your Tears                   The Weeknd   
81150                         Call Out My Name      